# Question 3: How do choices about data such as i) amount of training data and ii) balance of classes in a classification problem affect the performance of a neural network?

## Introduction

### Overview

In this final notebook we will look at how choices about the data affect the Neural Network's (NN) performance. I assume you have read and understood both the Question 1 and Question 2 notebooks before starting this one.

The first part of this will focus on chaning the ratio between the training, validation, and testing sets. In Question 2 I used a 64/16/20 split but here we will see what happens with a much smaller or larger training set. The second part will then focus on adjusting the weighting of the different classes to force the model to pay more or less attention to certain classes.